<a href="https://colab.research.google.com/github/marianasmoura/tecnicas-de-otimizacao/blob/main/Exercicios_Steepest_Descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


UNIVERSIDADE FEDERAL DO PIAUÍ

CURSO DE GRADUAÇÃO EM ENGENHARIA ELÉTRICA

DISCIPLINA: TÉCNICAS DE OTIMIZAÇÃO

DOCENTE: ALDIR SILVA SOUSA

DISCENTE: MARIANA DE SOUSA MOURA

Atividade 3: Otimização Irrestrita pelo Método de Newton Multivariável


Resolva os exercícios usando o método de Descida Gradiente.

# **Método da Descida Gradiente - Multivariável**

#**Exercícios**

Trecho responsável pela busca do lambda através do método da bisseção

In [1]:
import numpy as np
import sympy as sym   #Para criar variáveis simbólicas.

class Params:
  def __init__(self,f,vars,eps,a,b):
    self.f = f
    self.a = a
    self.b = b    
    self.vars = vars #variáveis simbólicas
    self.eps = eps

def eval(sym_f,vars,x):
   map = dict()
   map[vars[0]] = x
   return sym_f.subs(map)    

import pandas as pd
import math

def bissecao(params):
   n = math.ceil( -math.log(params.eps/(params.b-params.a),2) )
   f = params.f
   diff = sym.diff(f) #retorna a derivada simbólica de f
   
   a = params.a
   b = params.b

   for k in range(n):
           x = (b + a)/2
           
           fx  = eval(f,params.vars,x)  #Não é necessário. Somente para debug
           dfx = eval(diff,params.vars,x)  

           if (dfx == 0): break  # Mínimo encontrado. Parar.
           if (dfx > 0 ):
               #Passo 2
               b = x
           else:
               #Passo 3
               a = x  
   x = (a+b)/2        
   return x


Calcula o gradiente e permite a substituição de valores em variáveis nas funções

In [2]:
# Função para o cálculo do gradiente 
import sympy as sym   #Para criar variáveis simbólicas.
def gradiente_simbolico(funcao,variaveis):
    g1 = [sym.diff(funcao,x) for x in variaveis]
    return g1

# Função para substituição dos valores nas variáveis simbólicas
def eval_simbolica(f,variaveis,x):
   mp = dict()
   for i in range(len(variaveis)):
      mp[variaveis[i]] = x[i]
   return float(f.subs(mp))

def eval_gradiente(grad_simb,variaveis,x):
   g2 = [ eval_simbolica(f,variaveis,x) for f in grad_simb]
   return g2

# Função para substituição da expressão com o lambda nas variáveis simbólicas de uma função
def eval_lambda(f,variaveis,x):
   mp = dict()
   for i in range(len(variaveis)):
      mp[variaveis[i]] = x[i]
   return f.subs(mp)

Parâmetros que serõ utilizados pela função

In [3]:
import numpy as np
import sympy as sym 

class Parametros:
  def __init__(self,f,d1f,vars,m,eps,nmax):
    self.f = f
    self.d1f = d1f
    self.m = m
    self.eps = eps
    self.nmax = nmax
    self.vars = vars

Código para a otimização a partir da Descida Gradiente

In [4]:
import pandas as pd
import math

lmbd = sym.Symbol('lmbd')

def steepestDescent(p):
      f = p.f
      d1f = p.d1f
      m = p.m
      eps = p.eps
      vars= p.vars
      nmax = p.nmax
      k = 0
      v = [0 for i in vars]
      
      cols = ['x','grad(x)','lambda']    
      table = pd.DataFrame([], columns=cols)


      g = eval_gradiente(d1f,vars,m)
      tolerancia = np.linalg.norm(g)  # avalia o valor do módulo do gradiente no ponto
      
      while ((tolerancia > eps) & (k < nmax)):
      #while (tolerancia > eps):
         d = eval_gradiente(d1f,vars,m)
      
         for j in range(0,len(m)):
              v[j] = m[j] - lmbd*d[j]

         param = Params(f(v),[lmbd],eps,0,3)
         lmb = bissecao(param)
        
         for j in range(0,len(m)):
             m[j] = m[j] - lmb*d[j]
         valor = np.array(m)
         k = k + 1
         
         row = pd.DataFrame([[valor,d,lmb]],columns=cols)
         table = table.append(row, ignore_index=True) # concatena valores de cada iteração
        

         g = eval_gradiente(d1f,vars,m)
         tolerancia = np.linalg.norm(g) # calcula o valor da incertexa critério de parada
         print(k,'Derivada: ', g)
         print(k,'Solução: ', m)
         print(k,'Tolerância: ', tolerancia)
         print('=====================================================================')
      return m,table

**1.** Considere o seguinte problema:

Minimizar $\sum_{i=2}^{n} [100(x_i-x^2_{i-1})^2 + (1-x_{i-1})^2]$

Resolva para n = 5, 10, e 50. Iniciando do ponto $x_0 = [-1.2,1.0,-1.2,1.0,...]$

In [5]:
# Para n = 5

import numpy as np
import sympy as sym 

variaveis = list(sym.symbols("x:5"))

c = variaveis

def f1(c):
    fo = 0
    for i in range(1,5):
        fo = fo + 100*(c[i] - c[i-1]**2)**2 + (1 - c[i-1])**2
    return fo

x = []
for i in range(1,6):
     if (i%2 != 0):
       x.append(-1.2)
     else: 
       x.append(1)

eps = 1e-3
nmax = 500
d1f = gradiente_simbolico(f1(c),c)
p  = Parametros(f1,d1f,c,x,eps,nmax)
m,df = steepestDescent(p)

1 Derivada:  [-311.2609414050821, -135.48916022479534, -121.52078589354642, 16.313055351376534, -146.69422149658203]
1 Solução:  [-0.963134765625, 0.1298828125, -0.479736328125, 0.1298828125, -0.7166015625]
1 Tolerância:  389.60659386407815
2 Derivada:  [32.344427502331506, 103.91754146603932, -79.04925388766392, 23.2919039382602, -91.59979279350011]
2 Solução:  [-0.39319895982956155, 0.37797087443505006, -0.25722512348593796, 0.10001271603141504, -0.44799642059952016]
2 Tolerância:  164.39840870238746
3 Derivada:  [-24.87305260419642, -21.80554405653276, -14.701118422485173, 9.008686649215523, -42.96138972767231]
3 Solução:  [-0.47611314165926877, 0.11158069245423635, -0.054584213861799485, 0.040304466189683565, -0.21318249864352623]
3 Tolerância:  56.89561338803853
4 Derivada:  [5.985546417202201, 12.822683801661958, -9.024512395677563, -1.1288998252196762, -8.02727533787585]
4 Solução:  [-0.3759165186199033, 0.19942040850228093, 0.004636600095965497, 0.004014590771701111, -0.0401202

In [6]:
df

,x,grad(x),lambda
0,"[-0.963134765625, 0.1298828125, -0.47973632812...","[-215.5999999999999, 792.0, -655.5999999999999...",0.001099
1,"[-0.39319895982956155, 0.37797087443505006, -0...","[-311.2609414050821, -135.48916022479534, -121...",0.001831
2,"[-0.47611314165926877, 0.11158069245423635, -0...","[32.344427502331506, 103.91754146603932, -79.0...",0.002563
3,"[-0.3759165186199033, 0.19942040850228093, 0.0...","[-24.87305260419642, -21.80554405653276, -14.7...",0.004028
4,"[-0.39564427170394373, 0.15715814499582673, 0....","[5.985546417202201, 12.822683801661958, -9.024...",0.003296
...,...,...,...
495,"[0.9520402408991446, 0.9073545167570918, 0.821...","[0.06940093626047883, -0.2946462010220898, -0....",0.003296
496,"[0.9525530771683173, 0.906691511992256, 0.8224...","[-0.4667949685624535, 0.6034816703927959, -0.7...",0.001099
497,"[0.952262286971628, 0.9075026239779329, 0.8222...","[0.15881021941856943, -0.4429752924443733, 0.0...",0.001831
498,"[0.9526597604656509, 0.9070315544390083, 0.822...","[-0.3617892070039943, 0.42877796254117584, -0....",0.001099


In [7]:
m

[0.9524639712658601,
 0.907682517436897,
 0.8228282829665703,
 0.6767551684479244,
 0.4559335934954086]

In [8]:
np.array(m)

array([0.95246397, 0.90768252, 0.82282828, 0.67675517, 0.45593359])

In [9]:
# Para n = 10

import numpy as np
import sympy as sym 

variaveis = list(sym.symbols("x:10"))
c = variaveis

def f1(c):
    fo = 0
    for i in range(1,10):
        fo = fo + 100*(c[i] - c[i-1]**2)**2 + (1 - c[i-1])**2
    return fo

x = []
for i in range(1,11):
     if (i%2 != 0):
       x.append(-1.2)
     else: 
       x.append(1)

eps = 1e-3
nmax = 100
d1f = gradiente_simbolico(f1(c),c)
p  = Parametros(f1,d1f,c,x,eps,nmax)
m,df = steepestDescent(p)

1 Derivada:  [-311.2609414050821, -135.48916022479534, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, 64.00224724854343, 173.30654859542847]
1 Solução:  [-0.963134765625, 0.1298828125, -0.479736328125, 0.1298828125, -0.479736328125, 0.1298828125, -0.479736328125, 0.1298828125, -0.479736328125, 1.0966796875]
1 Tolerância:  440.13885306532853
2 Derivada:  [32.344427502331506, 103.91754146603932, -76.73085751243768, 22.866067573663948, -51.16193967003774, 22.866067573663948, -51.16193967003774, 39.51768451536306, -24.577235216666367, 84.60459004669234]
2 Solução:  [-0.39319895982956155, 0.37797087443505006, -0.25722512348593796, 0.12254547200609522, -0.25722512348593796, 0.12254547200609522, -0.25722512348593796, 0.12254547200609522, -0.5969279429599794, 0.7793459193198942]
2 Tolerância:  182.5844152190473
3 Derivada:  [-45.84945539178375, -44.71655507442835, -3.0442231217130393, 9.240988415872408, -18.94862246298731

In [10]:
df

,x,grad(x),lambda
0,"[-0.963134765625, 0.1298828125, -0.47973632812...","[-215.5999999999999, 792.0, -655.5999999999999...",0.001099
1,"[-0.39319895982956155, 0.37797087443505006, -0...","[-311.2609414050821, -135.48916022479534, -121...",0.001831
2,"[-0.499802907896328, 0.03546921188828961, -0.0...","[32.344427502331506, 103.91754146603932, -76.7...",0.003296
3,"[-0.24794432725298465, 0.2811045852219414, 0.0...","[-45.84945539178375, -44.71655507442835, -3.04...",0.005493
4,"[-0.3256358721459049, 0.07977184160430059, 0.0...","[19.286337447357656, 49.97932835502163, -15.28...",0.004028
...,...,...,...
95,"[0.7416108460069464, 0.5542171666120882, 0.308...","[0.972988321762557, -3.59513574433565, -0.4952...",0.001831
96,"[0.7500456339657983, 0.5558816072823005, 0.317...","[-1.7717380245875631, -0.3496178966763921, -1....",0.004761
97,"[0.7472875766205144, 0.5635372421346592, 0.318...","[1.5062670515043806, -4.180997380701484, -0.22...",0.001831
98,"[0.7539764347310997, 0.5635260618560282, 0.325...","[-2.0294490978486976, 0.0033921793535114375, -...",0.003296


In [11]:
m

[0.7521414361449623,
 0.5700702552358693,
 0.32604444390549414,
 0.11293191370356194,
 0.021890862521000656,
 0.010545501855342707,
 0.01021459674976703,
 0.010204324966187079,
 0.010004087113902847,
 0.00010008175430340227]

In [12]:
# Para n = 50

import numpy as np
import sympy as sym 

variaveis = list(sym.symbols("x:50"))
c = variaveis

def f1(c):
    fo = 0
    for i in range(1,50):
        fo = fo + 100*(c[i] - c[i-1]**2)**2 + (1 - c[i-1])**2
    return fo

x = []
for i in range(1,51):
     if (i%2 != 0):
       x.append(-1.2)
     else: 
       x.append(1)

eps = 1e-3
nmax = 100
d1f = gradiente_simbolico(f1(c),c)
p  = Parametros(f1,d1f,c,x,eps,nmax)
m,df = steepestDescent(p)

1 Derivada:  [-311.2609414050821, -135.48916022479534, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354645, 4.007166221737819, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, -121.52078589354642, 4.007166221737862, 64.00224724854343, 173.30

In [13]:
df

,x,grad(x),lambda
0,"[-0.963134765625, 0.1298828125, -0.47973632812...","[-215.5999999999999, 792.0, -655.5999999999999...",0.001099
1,"[0.2907240071249646, 0.6756765487571101, 0.009...","[-311.2609414050821, -135.48916022479534, -121...",0.004028
2,"[0.4191978721055577, 0.239288423962716, 0.1778...","[-70.16386012806791, 238.32610122104512, -91.7...",0.001831
3,"[0.5014384203050949, 0.24172384998448862, 0.04...","[-11.819553874572083, -0.35001771877826826, 19...",0.006958
4,"[0.49690716798147044, 0.25109426203824814, 0.0...","[0.9517953598751632, -1.9682670652409717, -5.4...",0.004761
...,...,...,...
95,"[0.8133073475604781, 0.6576729939006368, 0.438...","[-1.8144365804201072, 0.26447929985872065, -2....",0.002563
96,"[0.811729910383807, 0.6632973861505709, 0.4396...","[0.8614910234192621, -3.0716680874306803, -0.5...",0.001831
97,"[0.8163507496526964, 0.662579837807689, 0.4452...","[-1.8025673947972223, 0.2799121916613245, -2.1...",0.002563
98,"[0.8147220997938851, 0.6682344583145018, 0.446...","[0.8894599762254529, -3.0881767461206806, -0.4...",0.001831


In [14]:
m

[0.8193999026267168,
 0.6673673619440517,
 0.45171345780380207,
 0.20744691061780263,
 0.051029360329817426,
 0.012476745852166364,
 0.010254718718427525,
 0.010209275209893336,
 0.010208439455465072,
 0.010208424120869196,
 0.01020842383961724,
 0.010208423834460316,
 0.010208423834365788,
 0.010208423834364055,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364022,
 0.010208423834364015,
 0.010208423834363699,
 0.010208423834348288,
 0.010208423833601

**2.** Resolva:

Minimizar $(x_1 - x^3_2)^2 + 3(x_1 - x_2)^4$

In [15]:
import numpy as np
import sympy as sym 

x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')
variaveis = [x1,x2]

c = variaveis

fo = 0
def fo(c):
    return (c[0] - c[1]**3)**2 + 3*(c[0] - c[1])**4

x = [1.2,1.5]

eps = 1e-3
nmax = 500
d1f = gradiente_simbolico(fo(c),c)
p  = Parametros(fo,d1f,c,x,eps,nmax)
m,df = steepestDescent(p)

1 Derivada:  [0.12518138031716364, -0.22882191410531721]
1 Solução:  [1.266755126953125, 1.0760106811523438]
1 Tolerância:  0.2608253176992823
2 Derivada:  [0.04232431067070003, 0.021487096351164928]
2 Solução:  [1.2622166906794192, 1.0843065928331639]
2 Tolerância:  0.04746622571212269
3 Derivada:  [0.08106090606500524, -0.15969019390560496]
3 Solução:  [1.224010113847165, 1.0649099769253434]
3 Tolerância:  0.1790860924854002
4 Derivada:  [0.02485029017216174, 0.01304697856786572]
4 Solução:  [1.2208931478546752, 1.0710504079654148]
4 Tolerância:  0.02806707272572232
5 Derivada:  [0.052321832665950384, -0.09966363537189901]
5 Solução:  [1.1983149362151, 1.0591963233120183]
5 Tolerância:  0.11256293523654469
6 Derivada:  [0.017219445895577862, 0.00918986481213635]
6 Solução:  [1.1962264060293375, 1.0631745958665362]
6 Tolerância:  0.019518271752798074
7 Derivada:  [0.03800867686194444, -0.07120475574132842]
7 Solução:  [1.1803543313274931, 1.0547038110525175]
7 Tolerância:  0.080714167

In [16]:
df

,x,grad(x),lambda
0,"[1.266755126953125, 1.0760106811523438]","[-4.6739999999999995, 29.6865]",0.014282
1,"[1.2622166906794192, 1.0843065928331639]","[0.12518138031716364, -0.22882191410531721]",0.036255
2,"[1.224010113847165, 1.0649099769253434]","[0.04232431067070003, 0.021487096351164928]",0.902710
3,"[1.2208931478546752, 1.0710504079654148]","[0.08106090606500524, -0.15969019390560496]",0.038452
4,"[1.1983149362151, 1.0591963233120183]","[0.02485029017216174, 0.01304697856786572]",0.908569
5,"[1.1962264060293375, 1.0631745958665362]","[0.052321832665950384, -0.09966363537189901]",0.039917
6,"[1.1803543313274931, 1.0547038110525175]","[0.017219445895577862, 0.00918986481213635]",0.921753
7,"[1.1788093008837641, 1.057598242651866]","[0.03800867686194444, -0.07120475574132842]",0.040649
8,"[1.1601547571411717, 1.0485765156344575]","[0.013111799428512795, 0.006341140088083265]",1.422729
9,"[1.158866437286304, 1.0512405330191015]","[0.031132496315857974, -0.06437649089971201]",0.041382


In [17]:
m

[1.0743683154985644, 1.0242969819746126]

**3.** Resolva:



$2(x_1 - 2)^4 + (2x_1 - x_2)^2 = 4$


In [18]:
import numpy as np
import sympy as sym 

x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')
variaveis = [x1,x2]

c = variaveis

fo = 0
def fo(c):
    return (2*(c[0] - 2)**4 + (2*c[0] - c[1])**2 - 4)**2

x = [1.2,0.5]

eps = 1e-3
nmax = 300
d1f = gradiente_simbolico(fo(c),c)
p  = Parametros(fo,d1f,c,x,eps,nmax)
m,df = steepestDescent(p)

1 Derivada:  [0.060123472137406354, 0.01411795214220896]
1 Solução:  [0.8772600421875003, 0.8500033789062498]
1 Tolerância:  0.06175879269016826
2 Derivada:  [0.002155724219767421, 0.0005046112987580334]
2 Solução:  [0.8767976668524592, 0.8498948057879686]
2 Tolerância:  0.0022139962679566844
3 Derivada:  [7.057252525030156e-05, 1.651773667993772e-05]
3 Solução:  [0.8767810884069215, 0.849890925110256]
3 Tolerância:  7.247976921343094e-05


In [19]:
df

,x,grad(x),lambda
0,"[0.8772600421875003, 0.8500033789062498]","[3.0078335999999974, -3.261919999999998]",0.10730
1,"[0.8767976668524592, 0.8498948057879686]","[0.060123472137406354, 0.01411795214220896]",0.00769
2,"[0.8767810884069215, 0.849890925110256]","[0.002155724219767421, 0.0005046112987580334]",0.00769


In [20]:
m

[0.8767810884069215, 0.849890925110256]

**4** Resolva:

Minimizar $(x_1 - 3)^4 + (x_1 - 3x_2)^2$

In [21]:
import numpy as np
import sympy as sym 

x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')
variaveis = [x1,x2]

c = variaveis

def f4(c):
    return (c[0] - 3)**4 + (c[0] - 3*c[1])**2

x = [1.5,1]

eps = 1e-4
nmax = 300
d1f = gradiente_simbolico(f4(c),c)
p  = Parametros(f4,d1f,c,x,eps,nmax)
n,t = steepestDescent(p)

1 Derivada:  [-1.2189947589327108, -2.2427215576171875]
1 Solução:  [2.2107467651367188, 0.6123199462890625]
1 Tolerância:  2.552596365919665
2 Derivada:  [-1.5108617082256197, 0.8213843286490192]
2 Solução:  [2.323744097297916, 0.8202138284686953]
2 Tolerância:  1.7197021011595695
3 Derivada:  [-0.46010848025836104, -0.8479326027019987]
3 Solução:  [2.429492428835253, 0.7627234427949732]
3 Tolerância:  0.9647225053509657
4 Derivada:  [-0.7074229085276382, 0.38423439271131876]
4 Solução:  [2.474839118807005, 0.8462927280862972]
4 Tolerância:  0.8050361731325736
5 Derivada:  [-0.25850749496767733, -0.4756084367156568]
5 Solução:  [2.5293401299330434, 0.8166906857157001]
5 Tolerância:  0.5413220021665244
6 Derivada:  [-0.4290885798250921, 0.23323331388186297]
6 Solução:  [2.5554804065619314, 0.8647842085140806]
6 Tolerância:  0.4883797580168839
7 Derivada:  [-0.17072691690908037, -0.31397144880007843]
7 Solução:  [2.5901487419832856, 0.8459400557277575]
7 Tolerância:  0.35738739599893604

In [22]:
t

,x,grad(x),lambda
0,"[2.2107467651367188, 0.6123199462890625]","[-16.5, 9.0]",0.043076
1,"[2.323744097297916, 0.8202138284686953]","[-1.2189947589327108, -2.2427215576171875]",0.092697
2,"[2.429492428835253, 0.7627234427949732]","[-1.5108617082256197, 0.8213843286490192]",0.069992
3,"[2.474839118807005, 0.8462927280862972]","[-0.46010848025836104, -0.8479326027019987]",0.098557
4,"[2.5293401299330434, 0.8166906857157001]","[-0.7074229085276382, 0.38423439271131876]",0.077042
...,...,...,...
295,"[2.9316969712715397, 0.9772782897118157]","[-0.0007877291458786573, -0.0014746416112352279]",0.105515
296,"[2.931843529040471, 0.977200078292116]","[-0.0015504132275552251, 0.0008273871834454383]",0.094528
297,"[2.9319258850885888, 0.9773542383485642]","[-0.000779841925939273, -0.0014597649847409855]",0.105606
298,"[2.932070893019216, 0.9772767077756409]","[-0.0015355048863447962, 0.0008209797426239618]",0.094437


In [23]:
n

[2.9321524482645955, 0.9774292682346583]

**Conclusões**


Pode-se observar que ambos os métodos, Newton e Descida Gradiente, apresentam um enorme dificuldade para convergir quando se tem um grande número de variáveis, levando bastante tempo para convergir. Em ambos os casos, demorou bantante tempo para convergência quando o número de variáveis foi de 50. Para 10, o método atinge o máximo de iterações e não converge. 

O método da Descida Gradiente apresenta muitas mais iterações até a convergência se comparado ao método de Newton, porém tem a vantagem de convergir mesmo com pontos iniciais distante da solução. Além pode cair em um mínimo local mais facilmente e não apresentar uma precisão tão alta como no método de Newton. 

Conforme pode-se observar, para ambos os métodos, com exceção da questão 3, todos as demais questões convergiram para ótimos locais próximos. O método de Descida Gradiente apresentando mais iterações e precisão menor, conforme dito. A respeito de não ser obtido pontos de convergência próximos na questão 3 pode-se dever ao fato da função apresentar muitos pontos de inflexão. Se a aplicação exigir uma precisão da ordem de $ 10^4 $, torna-se mais apropriado o uso do método de Newton.